# machince learning tool SVCM, PCA, GBM 
!pip install scikit-learn

In [1]:
# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# scikit learn
# from sklearn.datasets import load_iris
import sklearn.datasets as sk_dataset

# split learning / test
from sklearn.model_selection import train_test_split

In [3]:
# in/out data load

In [4]:
iris_data = sk_dataset.load_iris()

In [5]:
iris_x = iris_data.data # input 
iris_y = iris_data.target # output(target)

In [6]:
type(iris_x), iris_x.shape # sample # 붓꽃의 꽃받침 길이, 꽃받침 너비 과 꽃잎의 길이, 꽃잎의 너비 에 대한 값

(numpy.ndarray, (150, 4))

In [7]:
type(iris_y), iris_y.shape, iris_y[100:110] # class number 0- setosa, 1 - versicolor, 2 - virginica

(numpy.ndarray, (150,), array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2]))

In [8]:
iris_x[:, 1].min(), iris_x[:, 1].max()  # input-out normalizing 0~1

(2.0, 4.4)

In [9]:
# norm 0~1 .. input mapping 
def norm_maxmin(data, out_min=0, out_max=255):
    gradient = (out_max - out_min) / (np.max(data) - np.min(data))
    bias = out_min - np.min(data) * gradient
    out = data * gradient + bias

    return out

In [10]:
for it_col in range(iris_x.shape[1]):
    iris_x[:, it_col] = norm_maxmin(iris_x[:, it_col], 0, 1)

In [11]:
iris_x[:, 0].min(), iris_x[:, 0].max()  # input-out normalizing 0~1

(0.0, 1.0)

In [12]:
iris_x.shape, iris_y.shape

((150, 4), (150,))

In [13]:
# train set, test set
x_train, x_test, y_train, y_test = train_test_split(iris_x, iris_y, test_size=0.1, shuffle=True)

In [14]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((135, 4), (15, 4), (135,), (15,))

In [15]:
y_train, y_test

(array([1, 1, 0, 2, 0, 2, 1, 2, 1, 1, 1, 1, 0, 1, 0, 2, 1, 0, 2, 0, 0, 1,
        2, 1, 2, 2, 0, 0, 2, 0, 0, 1, 1, 1, 0, 0, 0, 2, 0, 0, 1, 2, 2, 0,
        2, 1, 0, 1, 0, 1, 2, 2, 1, 2, 2, 2, 2, 1, 0, 1, 2, 1, 0, 2, 1, 0,
        1, 2, 2, 1, 1, 0, 0, 1, 0, 0, 1, 0, 2, 1, 2, 0, 0, 1, 1, 0, 1, 0,
        1, 2, 1, 2, 1, 0, 1, 0, 2, 2, 0, 2, 1, 0, 2, 1, 1, 0, 1, 0, 2, 2,
        0, 1, 0, 2, 1, 1, 0, 2, 2, 0, 0, 1, 2, 2, 0, 2, 1, 2, 1, 1, 2, 2,
        0, 1, 2]),
 array([2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 1, 1, 0, 0]))

In [16]:
# mini batch  #norm, mini
batch_size = 15

In [17]:
x_train.shape # ?, 15, 4

(135, 4)

In [18]:
x_train = x_train.reshape(-1, batch_size, x_train.shape[1])
y_train = y_train.reshape(-1, batch_size, )

In [19]:
type(x_train), y_train.shape # ?, 15, 4

(numpy.ndarray, (9, 15))

In [20]:
# 자료형 
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)

In [21]:
type(x_train), type(y_train), type(x_test), type(y_test), x_train.dtype, y_train.dtype

(torch.Tensor,
 torch.Tensor,
 torch.Tensor,
 torch.Tensor,
 torch.float32,
 torch.int64)

# Model init

In [22]:
class Iris_fcn(nn.Module):
    
    def __init__(self, model_input, model_output):
        super(Iris_fcn, self).__init__()
        
        # layer
        self.fc = nn.Linear(model_input, 16) # learning weight, bias
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, model_output)
        
        #activation 
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        out = self.fc(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        
       # Cross Entropy Loss --> softmax 생략
        
        return out

In [23]:
model = Iris_fcn(x_train.shape[2], 3)

In [24]:
from torchsummary import summary

summary(model, input_size=((1, 4)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 16]              80
              ReLU-2                [-1, 1, 16]               0
            Linear-3                 [-1, 1, 8]             136
              ReLU-4                 [-1, 1, 8]               0
            Linear-5                 [-1, 1, 3]              27
Total params: 243
Trainable params: 243
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


# Model train

In [25]:
learning_rate = 1e-2 #0.01

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
epochs = 1000
for _epoch in range(epochs):
    for _batch in range(x_train.shape[0]):
        optimizer.zero_grad()
        
        output = model(x_train[_batch, ])
        
        # 오차 계산 및 오차 역전파
        loss = criterion(output, y_train[_batch, ])
        loss.backward()

        #기울기 적용
        optimizer.step()
        
    if (_epoch+1) % 10 == 0:
        print(f'epoch:{_epoch}, loss:{loss.item()}')
        
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0

            outputs = model(x_test)
            pred = torch.argmax(outputs, axis=1)

            correct = (pred == y_test).sum()
            total = y_test.shape[0]

            acc = correct/total * 100
            print(f'accuracy : {acc}, correct:{correct}, total{total}')
            
            model.train()

epoch:9, loss:0.039494168013334274
accuracy : 100.0, correct:15, total15
epoch:19, loss:0.007129103410989046
accuracy : 100.0, correct:15, total15
epoch:29, loss:0.0033926726318895817
accuracy : 93.33333587646484, correct:14, total15
epoch:39, loss:0.002195026958361268
accuracy : 93.33333587646484, correct:14, total15
epoch:49, loss:0.0016481330385431647
accuracy : 93.33333587646484, correct:14, total15
epoch:59, loss:0.0013525326503440738
accuracy : 100.0, correct:15, total15
epoch:69, loss:0.001174574950709939
accuracy : 100.0, correct:15, total15
epoch:79, loss:0.0010553537867963314
accuracy : 100.0, correct:15, total15
epoch:89, loss:0.0009834575466811657
accuracy : 100.0, correct:15, total15
epoch:99, loss:0.0009304214618168771
accuracy : 100.0, correct:15, total15
epoch:109, loss:0.0008822563686408103
accuracy : 100.0, correct:15, total15
epoch:119, loss:0.0008552550571039319
accuracy : 100.0, correct:15, total15
epoch:129, loss:0.0008359296480193734
accuracy : 100.0, correct:15,